<a id='top'></a>
# PDRs4All: Creating NIRCam Imaging Data
---
**Author**: Amélie Canin (amelie.canin@irap.omp.eu) 

**Latest Update**: 15 juin 2022

This notebook follows MIRAGE example from STScI's available on the [Git](https://github.com/spacetelescope/mirage) and used some documentations (see [documentation](#doc) to find useful links).

<div class="alert alert-block alert-info">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Create a PDRs4all NIRCam image using MIRAGE and a image mosaic as input. We will:</ul>
    <ul>    
      <li>create and understand YAML files; </li>
      <li>create a seed image and the dark current corresponding; </li>
      <li>create the final observation. </li>    
    </ul>
</div>

## Introduction

<img align="left" width=15% src="pdrs4all_logo.png">

### The PDRs4All: Radiative feeback from massive stars

PDRs4All is a program of imaging and spectroscopy in the Orion Bar to understand the impact of massive stars. 
Massive stars disrupt their natal molecular cloud material by dissociating molecules, ionizing atoms and molecules, and heating the gas and dust. These processes drive the evolution of interstellar matter in our Galaxy and throughout the Universe from the era of vig- orous star formation at redshifts of 1-3, to the present day. Much of this interaction occurs in Photo-Dissociation Regions where far-ultraviolet photons of these stars create a largely neutral, but warm region of gas and dust.

PDRs4All has 12 NIRCam imaging filters in the same field of view to observe different Polycyclic Aromatic Hydrocarbons on the Orion Bar in the field of view on the [figure below](#fov). More detailled of the PDRs4All are available in the [Berné, Habart, Peeters et al. 2022](https://iopscience.iop.org/article/10.1088/1538-3873/ac604c) and on the [website](https://pdrs4all.org).

<a id='fov'></a>
<img width=30% src="spitzer_fov.jpeg">

In this notebook, we will simulate one NIRCam observation for the filter F140M and demonstrate the whole processus to create raw images using MIRAGE (Multi-Instrument RAmp GEnerator).

<a id='doc'></a>
## Resources and Documentation

There are several places to find information on intalling and running MIRAGE. This notebook will provides examples of running MIRAGE with a background image, but will not demonstrate all options and features. Please see the following links for more in-depth documentations.

* [JWST Documentation (JDox) for MIRAGE](https://jwst-docs.stsci.edu/jwst-other-tools/mirage-jwst-data-simulator)

* [MIRAGE Documentation](https://mirage-data-simulator.readthedocs.io/en/latest/)

* [`mirage` package GitHub repository](https://github.com/spacetelescope/mirage)

* [Help Desk](https://stsci.service-now.com/jwst?id=sc_category&sys_id=e15706fc0a0a0aa7007fc21e1ab70c2f): If you have any questions or problems regarding MIRAGE or others, submit a ticket to the Help Desk

### Installation

<div class="alert alert-block alert-info">
    Before running this notebook, you will have to first install the <code>mirage</code> package.
    
**NOTE:** The `mirage` package requries Python 3.8+ <br><br>
    
The recommended way to install the pipeline is via `pip`. Follow the steps below to create a new conda environment, activate that environment, and then install the latest released version of the mirage. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install mirage`

For more detailed instructions on the various ways to install the package, including installing more recent development versions of mirage, see the [installation instructions](https://mirage-data-simulator.readthedocs.io/en/latest/install.html?highlight=git) on GitHub.
    
</div>

### Reference Files

Reference files are used to simulated the NIRCam images, to create the dark current etc. The same reference files used in the Calibration Pipeline. Others files are specific to MIRAGE.

When running MIRAGE, it will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.
    
<div class="alert alert-block alert-info">
    
You will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are three environment variables that should be set prior to calling MIRAGE. These are the `CRDS_PATH`, `CRDS_SERVER_URL` and `MIRAGE_DATA` variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
>`$ export MIRAGE_DATA=$HOME/mirage_data`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>
`os.environ["MIARGE_DATA"] = "/user/myself/mirage_data"`<br>

The first time you run MIRAGE, the CRDS server should download all of the context and reference files that are needed. 
</div>

<strong>Note:</strong>The <code>MIRAGE_DATA</code> directory requires a lot of space in your disk.</div>

In [ ]:
import os

# Make sure to replace with the path to your CRDS cache and mirage data directories
os.environ["MIRAGE_DATA"] = "/path/to/mirage/data/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
os.environ["CRDS_PATH"] = "/path/to/crds/files/"
os.environ["CRDS_CONTEXT"] = "jwst_0862.pmap"


The following cell may or may not be needed depending on the latest version of MIRAGE.

In [ ]:
# Installing additional modules

import sys
!{sys.executable} -m pip install jwst-gtvt -q
import jwst_gtvt

## Imports

In [ ]:
import yaml

# Steps mirage
from mirage.dark.dark_prep import DarkPrep
from mirage.ramp_generator.obs_generator import Observation
from mirage.seed_image.fits_seed_image import ImgSeed
from mirage.yaml import yaml_generator

import jwst
from astropy.io import fits


Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Use this version for non-interactive plots (easier scrolling of the notebook)
#%matplotlib inline

# Use this version if you want interactive plots
#%matplotlib notebook
%matplotlib inline

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
#%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

# You may want to change the following configurations to customize 
# figure sizes and resolutions
rcParams['figure.figsize'] = [11,8]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

Check which version of mirage you are running

In [ ]:
import mirage
print(mirage.__version__)

Set the directories for outputs

In [ ]:
output_dir = './output2/f140m'
simdata_dir = os.path.join(output_dir,'stage0')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Set the path of the image used in background. You can download it here:  
[Orion_bar.fits](https://stsci.box.com/s/k0ilbd562rft77v9lq2to4b1qmt9fm3h)

In [ ]:
mosaic_file = './Orion_bar.fits'

Set files containing observation information as filters used, number of groups, dithers, etc.
The xml and pointing files can be downloaded here:  
[ERS1288_nircam.xml](https://stsci.box.com/s/1si8otf9rtqnyvzbz3dajveyayz5193m)  
[ERS1288_nircam.pointing](https://stsci.box.com/s/keb9hvimsrmxzud67qjbry2d7hghejsx)

<div class="alert alert-block alert-info">

The `xml` and `pointing` files can be found in APT.
    
To do this, in APT, when the observation is open, go to `File/Export` and then select `xml`. To export `pointing` you have to do the same one more time selecting `pointing`.
</div>


In [ ]:
xml_file = './ERS1288_nircam.xml'
pointing_file = xml_file.replace('.xml', '.pointing')

Function to show the different images

In [ ]:
def show(array,title,min=0,max=1000):
    plt.figure(figsize=(10,10))
    plt.imshow(array,clim=(min,max),origin='lower')
    plt.title(title)
    plt.colorbar()

## Mosaic image

We use a mosaic image as background for the short wavelength: the Hubble Space Telescope image of the Orion Nebula at 1.4 μm obtained by [Robberto et al 2020](https://iopscience.iop.org/article/10.3847/1538-4357/ab911e) with the Wide Field Camera 3 (WFC3).

Store the mosaic image in the variable data

In [ ]:
data = fits.getdata(mosaic_file)

Plot the image

In [ ]:
show(data,'HST Mosaic',max=200)

## YAML generator

**Description**

The first stage of the MIRAGE simulation is the creation of YAML files from the inputs above, especially the files exported from APT. YAML is a language used for configuration files as XML or JSON. The YAML files contain all the information needed for MIRAGE to work properly and simulate the right observations like the telescope and instrument set- tings, the reference files. The YAML generator allows to execute this first step. The YAML generator takes as inputs the xml and pointing files created with APT, the roll angle, the datatype set as raw to obtain the same results as the JWST, and the output directories.

**Inputs**

* Files from APT (`.xml` and `.pointing`) containing the observation information
* Output directory where to store the yaml files
* Simulation data output directory (`simdata_output_dir`) where to store the MIRAGE simulation
* Roll angle corresponding to the telescope V3 angle

**Outputs**

* YAML files (`.yaml`) resulting from yaml generation one for each detector, each filter and each dither


**Roll angle**

The roll angle of the telescope is needed so that MIRAGE can position the field of view correctly on the sky. It corresponds to the position angle of the V3 axis in degrees east of north.

In [ ]:
roll_angle = yaml_generator.all_obs_v3pa_on_date(pointing_file, date='2022-09-10')
print(roll_angle)

**YAML generator**

In [ ]:
yam = yaml_generator.SimInput(xml_file, pointing_file, verbose=True,
                            output_dir=output_dir,
                            simdata_output_dir=simdata_dir,
                            roll_angle=roll_angle,
                            datatype='raw'
                            )
yam.use_linearized_darks = True
yam.create_inputs()

We chose one observation.

In [ ]:
yamlfile = './output2/f140m/jw01288001001_01101_00021_nrcb1.yaml'

Open of the yaml into a dictionnaire

In [ ]:
with open(yamlfile) as f:
    yaml_dict = yaml.safe_load(f)
yaml_dict

Find the filter of the observation

In [ ]:
yaml_dict['Readout']['filter']

## Simulation

**Description**

The JWST Multi Instrument Ramp Generator (MIRAGE), is a simulator to create JWST-like NIRCam images. It is an open source Python package developed by the STScI [Hilbert et al. 2019](https://zenodo.org/record/3519262#.Yqss8C0itUc).

This simulator is divided in three stages: creation of the seed image; creation of the dark current; creation of the final observation. See the figures on this [page](https://mirage-data-simulator.readthedocs.io/en/latest/three_steps.html).

The simulation needs an YAML file corresponding to the observation, containing the filter, the detector, the roll angle among others. This simulator can be used with a stellar catalog or an image used in background.

In this notebook, we will only present the method using an image as background.


**Inputs**

* YAML file (`.yaml`) resulting from yaml generation

**Outputs**

* A raw exposure (`*uncal.fits`) containing the 4-dimensional raw data from all detector readouts: (nintegrations x ngroups x ncols x nrows)



### Seed image

The seed image corresponds to an idealized, noiseless image of the scene. As our extended images are bigger than the detectors, MIRAGE crops the scene.

**Inputs**

* YAML file (`.yaml`) resulting from yaml generation
* An image used in background 

**Outputs**

* A seed image (`*botted_seed_image.fits`)

In [ ]:
# Define output filenames and directories
sim_data_dir = yaml_dict['Output']['directory']
simulated_filename = yaml_dict['Output']['file']
crop_file = simulated_filename.replace('.fits', '_cropped_from_mosaic.fits')
crop_file = os.path.join(sim_data_dir, crop_file)
blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')

In [ ]:
# Instantiate the seed image
seed = ImgSeed(paramfile=yamlfile, mosaic_file=mosaic_file, 
               cropped_file=crop_file,
               outdir=sim_data_dir, blotted_file=blot_file, mosaic_fwhm=0.009,
               mosaic_fwhm_units='arcsec', gaussian_psf=True)

In [ ]:
# Creation of the seed image
seed.crop_and_blot()

In [ ]:
# Shape of the seed image
seed.seed_image.shape

In [ ]:
# Plot the seed image
show(seed.seed_image,'Seed image',max=200)

### Dark current

This stage prepares the dark current exposure for the observation.

**Inputs**

* YAML file (`.yaml`) resulting from yaml generation

**Outputs**

* A dark current (`*dark_prep_object.fits`)

In [ ]:
dark = DarkPrep()

In [ ]:
dark.paramfile = yamlfile
dark.prepare()

### Observation

This stage produces the final raw image with the noise from the background but also due to the detectors. This stage combines the seed im- age and the dark current exposure.

**Inputs**

* YAML file (`.yaml`) resulting from yaml generation
* A seed image (`*botted_seed_image.fits`)
* A dark current (`*dark_prep_object.fits`)

**Outputs**

* A raw exposure (`*uncal.fits`) containing the 4-dimensional raw data from all detector readouts: (nintegrations x ngroups x ncols x nrows).

In [ ]:
# Instantiate the observation
obs = Observation()

In [ ]:
# Define parameters of the observation
obs.paramfile = yamlfile    
obs.seed = seed.seed_image
obs.segmap = seed.seed_segmap
obs.seedheader = seed.seedinfo
obs.linDark = dark.prepDark

In [ ]:
# Creation of the observation
obs.create()

In [ ]:
# Path of the observation
obs.raw_output

In [ ]:
# Shape of the observation: (nintegrations x ngroups x ncols x nrows)
obs.raw_outramp.shape

In [ ]:
# Plot of one group of the observation
show(obs.raw_outramp[0,0,:,:],'Observation',max=20000)